<a href="https://colab.research.google.com/github/marcos-da-luz-pinto/TSFMs/blob/main/Resultados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# 1. Carregar os arquivos
df_arima = pd.read_csv('RESULTADOS AutoARIMA')
df_chronos = pd.read_csv('RESULTADOS_Chronos.csv')
df_moirai = pd.read_csv('RESULTADOS_Moirai.csv')

# 2. Padronizar e Unificar
df_arima['Modelo'] = 'AutoARIMA'
df_all = pd.concat([df_arima, df_chronos, df_moirai], ignore_index=True)

def gerar_tabela_por_H(df, valor_h):
    # Filtrar apenas o horizonte H solicitado
    df_h = df[df['H'] == valor_h]

    # Criar a Pivot Table: Linhas=Variável, Colunas=Modelo e C
    tabela = df_h.pivot_table(
        index='Variável',
        columns=['Modelo', 'C'],
        values='MASE (Mediana)'
    )

    # Ordenar as colunas para que os modelos fiquem agrupados e o C em ordem crescente
    tabela = tabela.sort_index(axis=1, level=[0, 1])
    return tabela

# 3. Gerar as duas tabelas solicitadas
tabela_mase_h12 = gerar_tabela_por_H(df_all, 12)
tabela_mase_h24 = gerar_tabela_por_H(df_all, 24)

# 4. Salvar os resultados
with pd.ExcelWriter('TABELAS_MASE_POR_HORIZONTE.xlsx') as writer:
    tabela_mase_h12.to_excel(writer, sheet_name='MASE_H12')
    tabela_mase_h24.to_excel(writer, sheet_name='MASE_H24')

# 5. Exibição rápida no console
print("--- TABELA MASE (H=12) ---")
print(tabela_mase_h12.head())

print("\n--- TABELA MASE (H=24) ---")
print(tabela_mase_h24.head())

--- TABELA MASE (H=12) ---
Modelo                   AutoARIMA                 Chronos-Base          \
C                              24      60      120          24      60    
Variável                                                                  
CUB Desp. Administrativa    0.0019  0.0019  0.0030       0.0040  0.0029   
CUB Equipamento             0.0027  0.0032  0.0030       0.0023  0.0037   
CUB Global                  0.0030  0.0048  0.0100       0.0035  0.0051   
CUB Mão de Obra             0.0032  0.0052  0.0033       0.0014  0.0033   
Cub Material                0.0062  0.0119  0.0204       0.0050  0.0081   

Modelo                           Moirai-Base                  
C                            120         24      60      120  
Variável                                                      
CUB Desp. Administrativa  0.0040      0.0174  0.0168  0.0308  
CUB Equipamento           0.0034      0.0073  0.0177  0.0277  
CUB Global                0.0069      0.0163  0.0936  0.2

In [3]:
import pandas as pd

# 1. Carregar os arquivos
df_arima = pd.read_csv('RESULTADOS AutoARIMA')
df_chronos = pd.read_csv('RESULTADOS_Chronos.csv')
df_moirai = pd.read_csv('RESULTADOS_Moirai.csv')

# 2. Padronizar e Unificar
df_arima['Modelo'] = 'AutoARIMA'
df_all = pd.concat([df_arima, df_chronos, df_moirai], ignore_index=True)

def gerar_tabela_crps_por_H(df, valor_h):
    # Filtrar apenas o horizonte H solicitado
    df_h = df[df['H'] == valor_h]

    # Criar a Pivot Table: Linhas=Variável, Colunas=[Modelo, C]
    # Usando a métrica CRPS (Mediana)
    tabela = df_h.pivot_table(
        index='Variável',
        columns=['Modelo', 'C'],
        values='CRPS (Mediana)'
    )

    # Ordenar as colunas para agrupar por Modelo e depois por janela C (24, 60, 120)
    tabela = tabela.sort_index(axis=1, level=[0, 1])
    return tabela

# 3. Gerar as duas tabelas solicitadas para CRPS
tabela_crps_h12 = gerar_tabela_crps_por_H(df_all, 12)
tabela_crps_h24 = gerar_tabela_crps_por_H(df_all, 24)

# 4. Salvar os resultados em Excel e CSV
with pd.ExcelWriter('TABELAS_CRPS_POR_HORIZONTE.xlsx') as writer:
    tabela_crps_h12.to_excel(writer, sheet_name='CRPS_H12')
    tabela_crps_h24.to_excel(writer, sheet_name='CRPS_H24')

tabela_crps_h12.to_csv('TABELA_CRPS_H12.csv')
tabela_crps_h24.to_csv('TABELA_CRPS_H24.csv')

# 5. Exibição para conferência
print("--- TABELA CRPS (H=12) ---")
print(tabela_crps_h12.head())

print("\n--- TABELA CRPS (H=24) ---")
print(tabela_crps_h24.head())

--- TABELA CRPS (H=12) ---
Modelo                   AutoARIMA                   Chronos-Base           \
C                              24       60       120          24       60    
Variável                                                                     
CUB Desp. Administrativa    0.3767   0.6081   0.6592       0.7971   0.8175   
CUB Equipamento             0.3386   0.2686   0.2317       0.2648   0.3389   
CUB Global                 15.5179  13.4349  18.2493      19.6434  18.3863   
CUB Mão de Obra             9.8551   8.3830   4.9961       9.8777   8.4659   
Cub Material               12.9328  13.1423  13.3605      14.2959   8.6169   

Modelo                            Moirai-Base                      
C                             120         24        60        120  
Variável                                                           
CUB Desp. Administrativa   0.9593      3.5788    5.9229    7.7995  
CUB Equipamento            0.2459      0.8725    1.7279    2.2596  
CUB Glob

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare

# Carregamento dos dados
# Certifique-se de que os nomes dos arquivos estão corretos no seu ambiente
file_arima = 'RESULTADOS AutoARIMA'
file_chronos = 'RESULTADOS_Chronos.csv'
file_moirai = 'RESULTADOS_Moirai.csv'

df_arima = pd.read_csv(file_arima)
df_chronos = pd.read_csv(file_chronos)
df_moirai = pd.read_csv(file_moirai)

# Padronização e União
df_arima['Modelo'] = 'AutoARIMA'
df_all = pd.concat([df_arima, df_chronos, df_moirai], ignore_index=True)

def realizar_analise_estatistica(df, metrica):
    """
    Realiza o Teste de Friedman e calcula os postos médios e a Diferença Crítica.
    """
    # Criar tabela pivot: cada linha é um "bloco" (Variável + C + H)
    pivot = df.pivot_table(
        index=['Variável', 'C', 'H'],
        columns='Modelo',
        values=metrica
    ).dropna()

    # Teste de Friedman
    # H0: Não há diferença entre os modelos
    stat, p_value = friedmanchisquare(
        pivot['AutoARIMA'],
        pivot['Chronos-Base'],
        pivot['Moirai-Base']
    )

    # Cálculo dos Ranks (Postos) - Menor erro ganha o melhor rank (1)
    ranks = pivot.rank(axis=1, ascending=True)
    avg_ranks = ranks.mean().sort_values()

    # Cálculo da Diferença Crítica (CD) para o Teste de Nemenyi
    # Fórmula: CD = q_alpha * sqrt( (k * (k + 1)) / (6 * n) )
    k = 3  # Número de modelos
    n = len(pivot)  # Número de blocos (42 no seu caso)
    q_alpha = 2.344 # Valor crítico para k=3 e alpha=0.05

    cd = q_alpha * np.sqrt((k * (k + 1)) / (6 * n))

    return {
        'metrica': metrica,
        'estatistica': stat,
        'p_valor': p_value,
        'ranks_medios': avg_ranks,
        'cd': cd,
        'n': n
    }

# Execução para MASE e CRPS
resultados_mase = realizar_analise_estatistica(df_all, 'MASE (Mediana)')
resultados_crps = realizar_analise_estatistica(df_all, 'CRPS (Mediana)')

# Exibição dos Resultados
for res in [resultados_mase, resultados_crps]:
    print(f"\n--- ANÁLISE ESTATÍSTICA: {res['metrica']} ---")
    print(f"Número de Blocos (n): {res['n']}")
    print(f"Estatística de Friedman: {res['estatistica']:.4f}")
    print(f"p-valor: {res['p_valor']:.10f}")
    print(f"Diferença Crítica (CD) de Nemenyi: {res['cd']:.4f}")
    print("Ranks Médios (Postos):")
    print(res['ranks_medios'])
    print("-" * 40)


--- ANÁLISE ESTATÍSTICA: MASE (Mediana) ---
Número de Blocos (n): 42
Estatística de Friedman: 55.7939
p-valor: 0.0000000000
Diferença Crítica (CD) de Nemenyi: 0.5115
Ranks Médios (Postos):
Modelo
Chronos-Base    1.392857
AutoARIMA       1.690476
Moirai-Base     2.916667
dtype: float64
----------------------------------------

--- ANÁLISE ESTATÍSTICA: CRPS (Mediana) ---
Número de Blocos (n): 42
Estatística de Friedman: 60.0476
p-valor: 0.0000000000
Diferença Crítica (CD) de Nemenyi: 0.5115
Ranks Médios (Postos):
Modelo
Chronos-Base    1.50000
AutoARIMA       1.52381
Moirai-Base     2.97619
dtype: float64
----------------------------------------
